In [3]:
# @Time    : 2025/8/9 17:51
# @Author  : liuzhou
# @File    : 1_context_bot.py
# @software: PyCharm
import io
from typing import Annotated

from IPython.display import display
from dotenv import load_dotenv
from IPython.display import Image
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from matplotlib import pyplot as plt
from typing_extensions import TypedDict

load_dotenv()

# 定义state
class State(TypedDict):
    # 状态变量messages类型是list，更新方式是add_messages
    # add_messages是内置的一个办法，将新的消息追加在原列表后面
    messages: Annotated[list, add_messages]

# 创建Graph，状态机
graph_builder  = StateGraph(State)

# 定义模型
llm = init_chat_model("gpt-4o", model_provider="openai")

# 定义一个执行节点
# 输入是State，输出是系统回复
def chatbot(state: State):
    # 调用大模型，并返回消息（列表）
    return {"messages": [llm.invoke(state["messages"])]}

# 定义工作流，包含节点和边
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

# 可视化展示工作流
try:
    display(Image(data=graph.get_graph().draw_png()))
except Exception as e:
    print(e)